# 졸음시 카카오톡 전송 및 알림

In [10]:
import cv2
import tensorflow.keras
import numpy as np
import winsound as sd
import kakao_utils
import requests
import json
from gtts import gTTS
from IPython.display import Audio 
from keras.models import load_model

In [11]:
## kakao_code.json 파일 만들기
url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = "개인 API key"
redirect_uri = '지정한 redirect_uri'
authorize_code = "인가 받은 코드"

data = {
    'grant_type':'authorization_code',
    'client_id':rest_api_key,
    'redirect_uri':redirect_uri,
    'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()

# 토큰을 파일로 저장하기
if "access_token" in tokens:
    with open("kakao_token.json", "w") as fp:
        json.dump(tokens, fp)
        print("Tokens saved successfully")
else:
    print(tokens)

In [12]:
## json 저장 및 확인
with open("kakao_token.json","w") as fp:
    json.dump(tokens, fp)
    
print(tokens)

In [14]:
## 음성 파일 생성
kor_wav = gTTS('3초 이상 졸았습니다. 졸지마세요!!', lang = 'ko') 
kor_wav.save('./sleep_voice.wav')

display(Audio('./sleep_voice.wav', autoplay=True))

In [13]:
# 카카오톡 메시지로 '졸음 방지 베타파' 영상 링크를 전송 
# 스피커로 졸음 방지 음성을 출력
def send_music_link():
    KAKAO_TOKEN_FILENAME = "kakao_token.json"  
    KAKAO_APP_KEY =  "개인 API key"
    tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

    # 텍스트 메시지 보내기
    template = {
          "object_type": "text",
          "text": "당신은 3초 이상 졸았습니다. 졸지 마세요!!!!",
          "link": {
            "web_url": "https://www.youtube.com/watch?v=7Q2N7919o5o",
            "mobile_web_url": "https://www.youtube.com/watch?v=7Q2N7919o5o"
          },
          "button_title": "잠깨는 노래 듣기"
    }

    # 카카오 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('텍스트 메시지를 성공적으로 보냈습니다.')
    else:
        print('텍스트 메시지를 보내지 못했습니다. 오류메시지 : ', res.json())


## gTTS 음성파일 출력 함수
def sound():
    display(Audio('./sleep_voice.wav', autoplay=True))

## 이미지 전처리
def preprocessing(frame):
    # 사이즈 조정
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
    
    # 이미지 정규화
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1
    
    # 이미지 차원 재조정 - 예측을 위해 reshape 해줍니다.
    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))
    
    return frame_reshaped

## 학습된 모델 불러오기 teachable machine / cnn
# google teachable machine 모델
model_filename = '모델명'
model = tensorflow.keras.models.load_model(model_filename)

# CNN 모델
# model = load_weights(model_filename)

# 카메라 캡쳐 객체, 0=내장 카메라
capture = cv2.VideoCapture(0)

# 캡쳐 프레임 사이즈 조절
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

time_num= 0    
image_num = 0

sleep_cnt = 1 # 3초간 "졸림" 상태를 확인하기 위한 변수
while True:
    ret, frame = capture.read()
    time_num = time_num + 1
    if ret == True: 
        #print("졸음 감지 중")

    # 이미지 뒤집기
    frame_fliped = cv2.flip(frame, 1)
    # 이미지 출력
    cv2.imshow("VideoFrame", frame_fliped)

    #-- 캡쳐 프레임 간격 설정
    if time_num == 1:
        image_num = image_num + 1
        cv2.imwrite('저장할 사진 경로'+str(image_num)+'.png', frame_fliped) #-- 본인 편의에 맞게 경로 설정 및 이미지 이름 변경
        time_num = 0
    
    # 1초마다 검사하며, videoframe 창으로 아무 키나 누르게 되면 종료
    if cv2.waitKey(200) > 0: 
        break        
    
    # 데이터 전처리
    preprocessed = preprocessing(frame_fliped)

    # 예측
    prediction = model.predict(preprocessed)
    print(prediction)
    
    if prediction[0,0] < prediction[0,1]:
        print('sleep')
        sleep_cnt += 1
        
        # 졸린 상태가 3초간 지속되면 소리 & 카카오톡 보내기
        if sleep_cnt % 3 == 0:
            sleep_cnt = 1
            print('3초간 졸고 있네요!!!')
            sound()  # gTTS 출력함수
            send_music_link() # 카카오톡 메시지 전송(졸음 경고 메시지, 유튜브 링크 전송)
            break ## 1번만 알람이 오면 프로그램을 정지 시킴 (반복을 원한다면, 주석으로 막기!)
    else:
        print('wake')
        sleep_cnt = 1
    
# 카메라 객체 반환
capture.release() 
# 화면에 나타난 윈도우들을 종료
cv2.waitKey()
cv2.destroyAllWindows()